In [1]:
%pylab inline
import keras
import keras.backend as K

import matplotlib.pyplot as plt

import numpy as np

Populating the interactive namespace from numpy and matplotlib


/home/ubuntu/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# SOPH: review these params and update accordingly
img_rows = 28
img_cols = 28
channels = 1
num_classes = 10
img_shape = (img_rows, img_cols, channels)
latent_dim = 72

In [3]:
# build discriminator

DROP = .25

img = keras.Input(shape=img_shape)

conv = keras.Sequential([
    keras.layers.InputLayer(input_shape=img_shape),
    
    keras.layers.Conv2D(32, 3, padding="same", activation="elu"),
    keras.layers.Conv2D(32, 3, strides=2, padding="same", activation="elu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(DROP),
    
    keras.layers.Conv2D(32, 3, padding="same", activation="elu"),
    keras.layers.Conv2D(32, 3, strides=2, padding="same", activation="elu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(DROP),
    
    keras.layers.Conv2D(64, 3, padding="same", activation="elu"),
    keras.layers.Conv2D(64, 3, strides=2, padding="same", activation="elu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(DROP),
    
    keras.layers.Conv2D(64, 3, padding="same", activation="elu"),
    keras.layers.Conv2D(64, 3, strides=2, padding="same", activation="elu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(DROP),
    
    keras.layers.Flatten(),
])
conv.summary()
img_embedding = conv(img)

# Discriminator
validity = keras.layers.Dense(1, activation='sigmoid')(img_embedding)

# Recognition
q_net = keras.layers.Dense(32, activation='elu')(img_embedding)

label = keras.layers.Dense(num_classes, activation='softmax')(q_net)

disk = keras.Model(img, validity)

print("disk summary:")
disk.summary()

q = keras.Model(img, label)

print("q summary")
q.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 32)          9248      
__________

In [4]:
DROP = .25

generator = keras.Sequential([
    keras.layers.Dense(4*4*32, activation="elu", input_dim=latent_dim),
    keras.layers.Reshape((4, 4, 32)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(DROP),
    
    keras.layers.Conv2DTranspose(32, 3, strides=2, padding="same", activation="elu"),
    keras.layers.Conv2D(32, 3, padding="same", activation="elu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(DROP),
    
    keras.layers.Conv2DTranspose(32, 3, strides=2, padding="same", activation="elu"),
    keras.layers.Conv2D(32, 3, padding="valid", activation="elu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(DROP),
    
    keras.layers.Conv2DTranspose(32, 3, strides=2, padding="same", activation="elu"),
    keras.layers.Conv2D(32, 3, padding="same", activation="elu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(DROP),
    
    keras.layers.Conv2D(channels, 5, padding="same", activation="tanh"),
    
])


generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               37376     
_________________________________________________________________
reshape_1 (Reshape)          (None, 4, 4, 32)          0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 4, 4, 32)          128       
_________________________________________________________________
dropout_5 (Dropout)          (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 8, 8, 32)          9248      
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 8, 8, 32)          9248      
_________________________________________________________________
batch_normalization_6 (Batch (None, 8, 8, 32)          128       
__________

In [5]:
DROP = .25

conv = keras.Sequential([
    keras.layers.InputLayer(input_shape=img_shape),
    
    keras.layers.Conv2D(32, 3, strides=2, padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(DROP),
    
    keras.layers.Conv2D(32, 3, strides=2, padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(DROP),
    
    keras.layers.Conv2D(32, 3, strides=2, padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(DROP),
    
    keras.layers.Conv2D(64, 3, strides=2, padding="same"),
    keras.layers.BatchNormalization(),
#     keras.layers.Dropout(DROP),
    
    
    keras.layers.GlobalAvgPool2D(),
    keras.layers.Dropout(DROP),
])
conv.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 14, 14, 32)        320       
_________________________________________________________________
batch_normalization_9 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
dropout_9 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 7, 7, 32)          9248      
_________________________________________________________________
batch_normalization_10 (Batc (None, 7, 7, 32)          128       
_________________________________________________________________
dropout_10 (Dropout)         (None, 7, 7, 32)          0         
__________

In [6]:
def mutual_info_loss(c, c_given_x):
    """The mutual information metric we aim to minimize"""
    eps = 1e-8
    conditional_entropy = K.mean(- K.sum(K.log(c_given_x + eps) * c, axis=1))
    entropy = K.mean(- K.sum(K.log(c + eps) * c, axis=1))

    return conditional_entropy + entropy

In [7]:
optimizer = keras.optimizers.Adam(0.0002, 0.5)
losses = ['binary_crossentropy', mutual_info_loss]

# Build and the discriminator and recognition network

disk.compile(loss=['binary_crossentropy'],
    optimizer=optimizer,
    metrics=['accuracy'])

# Build and compile the recognition network Q
q.compile(loss=[mutual_info_loss],
    optimizer=optimizer,
    metrics=['accuracy'])

In [8]:
# The generator takes noise and the target label as input
# and generates the corresponding digit of that label
gen_input = keras.Input(shape=(latent_dim,))
img = generator(gen_input)

# For the combined model we will only train the generator
disk.trainable = False

# The discriminator takes generated image as input and determines validity
valid = disk(img)
# The recognition network produces the label
target_label = q(img)

# The combined model  (stacked generator and discriminator)
combined = keras.Model(gen_input, [valid, target_label])
combined.compile(loss=losses, optimizer=optimizer)

In [9]:
def sample_generator_input(batch_size):
    # Generator inputs
    sampled_noise = np.random.normal(0, 1, (batch_size, 62))


    # SOPH: this code generates random combinations of your labels so that it can generate randomized images
    # the following code should be removed and replaced with something that generates random values for your 40
    # labels
    sampled_labels = np.random.randint(0, num_classes, batch_size).reshape(-1, 1)
    sampled_labels = keras.utils.to_categorical(sampled_labels, num_classes=num_classes)

    return sampled_noise, sampled_labels


def sample_images(epoch):
    # SOPH: this'll be hard to adjust so consider just removing it or greatly simplifying it to start out.
    # this generates those image matrices.
    r, c = 10, 10

    fig, axs = plt.subplots(r, c)
    for i in range(c):
        sampled_noise, _ = sample_generator_input(c)
        label = keras.utils.to_categorical(np.full(fill_value=i, shape=(r,1)), num_classes=num_classes)
        gen_input = np.concatenate((sampled_noise, label), axis=1)
        gen_imgs = generator.predict(gen_input)
        gen_imgs = 0.5 * gen_imgs + 0.5
        for j in range(r):
            axs[j,i].imshow(gen_imgs[j,:,:,0], cmap='gray')
            axs[j,i].axis('off')
    fig.savefig("images/%05d.png" % epoch)
    plt.close()

def save_model():

    def save(model, model_name):
        model_path = "saved_model/%s.json" % model_name
        weights_path = "saved_model/%s_weights.hdf5" % model_name
        options = {"file_arch": model_path,
                    "file_weight": weights_path}
        json_string = model.to_json()
        open(options['file_arch'], 'w').write(json_string)
        model.save_weights(options['file_weight'])

    save(generator, "generator")
    save(discriminator, "discriminator")

In [ ]:
batch_size=128
sample_interval=50
epochs = 10000
smooth = .1

# SOPH: replace the following code block with code that loads celebA. 
# x_train should be [n_examples x pixels x pixels]
# y_train should be [n_examples x n_labels]
# Load the dataset
(X_train, y_train), (_, _) = keras.datasets.mnist.load_data()

# Rescale -1 to 1
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)
y_train = y_train.reshape(-1, 1)
# SOPH: this is the end of the loading code. You may have to replace/remove any of these lines

# Adversarial ground truths
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

for epoch in range(epochs):

    # ---------------------
    #  Train Discriminator
    # ---------------------

    # Select a random half batch of images
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]

    # Sample noise and categorical labels
    sampled_noise, sampled_labels = sample_generator_input(batch_size)
    gen_input = np.concatenate((sampled_noise, sampled_labels), axis=1)

    # Generate a half batch of new images
    gen_imgs = generator.predict(gen_input)

    # Train on real and generated data
    d_loss_real = disk.train_on_batch(imgs, valid*(1-smooth))
    d_loss_fake = disk.train_on_batch(gen_imgs, fake)

    # Avg. loss
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ---------------------
    #  Train Generator and Q-network
    # ---------------------

    g_loss = combined.train_on_batch(gen_input, [valid, sampled_labels])

    # Plot the progress

    # If at save interval => save generated image samples
    if epoch % sample_interval == 0:
        print ("%d [D loss: %.2f, acc.: %.2f%%] [Q loss: %.2f] [G loss: %.2f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[1], g_loss[2]))
        sample_images(epoch//sample_interval)

/home/ubuntu/miniconda3/lib/python3.6/site-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.90, acc.: 23.44%] [Q loss: 0.73] [G loss: 3.37]
50 [D loss: 0.43, acc.: 37.11%] [Q loss: 1.19] [G loss: 2.50]
100 [D loss: 0.43, acc.: 39.45%] [Q loss: 1.09] [G loss: 2.46]
150 [D loss: 0.39, acc.: 42.58%] [Q loss: 1.14] [G loss: 2.33]
200 [D loss: 0.37, acc.: 44.53%] [Q loss: 1.04] [G loss: 2.36]
250 [D loss: 0.51, acc.: 30.08%] [Q loss: 1.02] [G loss: 2.50]
300 [D loss: 0.35, acc.: 45.31%] [Q loss: 2.12] [G loss: 2.31]
350 [D loss: 0.34, acc.: 48.83%] [Q loss: 2.08] [G loss: 2.38]
400 [D loss: 0.35, acc.: 45.70%] [Q loss: 1.74] [G loss: 2.37]
450 [D loss: 0.34, acc.: 44.53%] [Q loss: 1.77] [G loss: 2.21]
500 [D loss: 0.34, acc.: 46.09%] [Q loss: 1.58] [G loss: 1.84]
550 [D loss: 0.33, acc.: 46.09%] [Q loss: 1.65] [G loss: 1.35]
600 [D loss: 0.34, acc.: 47.66%] [Q loss: 1.57] [G loss: 0.79]
650 [D loss: 0.32, acc.: 47.66%] [Q loss: 1.51] [G loss: 0.43]
700 [D loss: 0.32, acc.: 47.66%] [Q loss: 1.56] [G loss: 0.31]
750 [D loss: 0.30, acc.: 48.83%] [Q loss: 1.58] [G loss: 0

In [ ]:
batch_size=128
sample_interval=50
epochs = 1000
smooth = .1

for epoch in range(epochs):

    # ---------------------
    #  Train Discriminator
    # ---------------------

    # Select a random half batch of images
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]

    # Sample noise and categorical labels
    sampled_noise, sampled_labels = sample_generator_input(batch_size)
    gen_input = np.concatenate((sampled_noise, sampled_labels), axis=1)

    # Generate a half batch of new images
    gen_imgs = generator.predict(gen_input)

    # Train on real and generated data
    d_loss_real = disk.train_on_batch(imgs, valid)
    d_loss_fake = disk.train_on_batch(gen_imgs, fake)

    # Avg. loss
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


    # Plot the progress

    # If at save interval => save generated image samples
    if epoch % sample_interval == 0:
        
         # ---------------------
        #  Train Generator and Q-network
        # ---------------------

        g_loss = combined.train_on_batch(gen_input, [valid, sampled_labels])
        
        print ("%d [D loss: %.2f, acc.: %.2f%%] [Q loss: %.2f] [G loss: %.2f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[1], g_loss[2]))
        sample_images(epoch//sample_interval)